In [1]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Milvus
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
import os
import ipywidgets as widgets
from IPython.display import display
from dotenv import load_dotenv

In [2]:
load_dotenv()  # take environment variables from .env.
openai_api_key = os.getenv('OPENAI_API_KEY')
milvus_api_key = os.getenv('MILVUS_API_KEY')

connection_args={
        "uri": "https://in03-5052868020ac71b.api.gcp-us-west1.zillizcloud.com",
        "user": "vaclav@pechtor.ch",
        "token": milvus_api_key,
        "secure": True
    }


In [3]:
embeddings = OpenAIEmbeddings()

milvus = Milvus(embedding_function=embeddings, collection_name='Simple_DIIN_QA', connection_args=connection_args)

In [4]:
query = "Warum nutzt die Informatik SAFe?"
print(f"Query: {query}")
docs = milvus.similarity_search(query)

print(docs)

Query: Warum nutzt die Informatik SAFe?
[Document(page_content='Die Dienststelle Informatik stellt sich in verschiedenen Bereichen auf ein sich stark änderndes Umfeld ein. Angetrieben durch die Digitalisierung richtet sie sich nach neuen Betriebsmodellen und Technologien aus. Dies umfasst auch die Auseinandersetzung mit dem Einsatz von agilen Methoden in neuen Arbeitsumgebungen. Um die lean-agile Arbeitsweise zu verankern, macht sich die Dienststelle dem Scaled Agile Framework (SAFe) als Modell für die Organisation zu Nutze. In mehreren Agile Teams, die in einem Agile Release Train organisiert sind, arbeitet die Dienststelle an den künftigen Lösungen für ihre Kundinnen und Kunden. \nSAFe\xa0stellt eine Reihe von Unternehmens- und Workflow-Mustern zur Implementierung von agilen Praktiken im gesamten Unternehmen bereit. Die Dienststelle Informatik orientiert sich unter anderem an strukturierten Leitlinien zu Rollen und Zuständigkeiten sowie zur Planung und Verwaltung von Aufgaben, die es

In [5]:
from langchain.prompts import PromptTemplate
language = "Swiss German Dialect"
prompt_template = f"""You are an assistant that answers questions about the Kanton Luzern,
 based on given information. Only use the information that was provided below.
 Use the following pieces of context to answer the question at the end.
 If you don't know the answer, just say that you don't know, don't try to make up an answer.

{{context}}

Question: {{question}}
Answer in {language}:"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}

In [6]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-4-0613',temperature=0.0)
qa = RetrievalQA.from_chain_type(llm=llm, 
                                 retriever=milvus.as_retriever(),
                                 chain_type='stuff',
                                 return_source_documents=True,
                                 chain_type_kwargs=chain_type_kwargs)
result = qa({"query": query})

In [7]:
print(f'Query: {query}')
print(f'Result: {result}')

Query: Warum nutzt die Informatik SAFe?
Result: {'query': 'Warum nutzt die Informatik SAFe?', 'result': "Die Informatik nutzt SAFe, um sich uf neui Betriebsmodelle und Technologie uszrichte. Mit SAFe chöi si agili Methode in neui Arbeitsumgebige iiführe und so ihri Leistige für ihri Chunde optimiere. Es hilft ihne au, ihri Rolle und Verantwortigkeite besser z'plane und z'verwaltige. D'Ziel isch, qualitativ hochstehendi Leistige so schnell wie möglich z'erbringe.", 'source_documents': [Document(page_content='Die Dienststelle Informatik stellt sich in verschiedenen Bereichen auf ein sich stark änderndes Umfeld ein. Angetrieben durch die Digitalisierung richtet sie sich nach neuen Betriebsmodellen und Technologien aus. Dies umfasst auch die Auseinandersetzung mit dem Einsatz von agilen Methoden in neuen Arbeitsumgebungen. Um die lean-agile Arbeitsweise zu verankern, macht sich die Dienststelle dem Scaled Agile Framework (SAFe) als Modell für die Organisation zu Nutze. In mehreren Agile Te